# Example project work, refactored

Now with functions in external module...

In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mikeio
import modelhelpers.modelutils as mu   # now a package

In [ ]:
fldr = Path("../data")
save_png = False

## 1. Plot stations on map

In [ ]:
stations = mu.get_stations(fldr)
station_names = stations.index
stations

In [ ]:
domain = mikeio.Mesh(fldr / "input" / "SouthernNorthSea.mesh")
mu.plot_mesh_and_stations(domain, stations, save_png=save_png);

## 2. Load observations and remove outliers 

In [ ]:
observations = {}
for stn in station_names:
    observations[stn] = mikeio.read(fldr / "obs" / f"{stn}.dfs0").to_dataframe()

In [ ]:
observations.keys()

Our outlier detector will mark an observation as an outlier if: 

* Value is out of range [0, 10] - non-physical values
* The gradient is too large (single outliers)

In [ ]:
detector = mu.get_anomaly_detector()
detector

Apply outlier detector for all observation series

In [ ]:
for stn, df in observations.items():
    mu.detect_and_remove_outliers(detector, stn, df, plot=True)
    if save_png:
        plt.savefig(f"Outliers_{stn}.png")

## 3. Compare with models

In [ ]:
cc = mu.compare_all(fldr, stations, observations)
sims = cc.mod_names

In [ ]:
cc.skill().round(3)

In [ ]:
for stn in station_names:
    cc[stn].plot_timeseries(figsize=(10,5))
    plt.legend(loc='upper left')
    if save_png:
        plt.savefig(f"Timeseries_comparison_{stn}.png")

## 4. Find best model 

In [ ]:
meanskill = cc.mean_skill()
meanskill.round(3)

In [ ]:
meanskill.plot_bar("rmse", title="Mean skill RMSE")
plt.ylabel("Hm0 [m]");